In [14]:
import pandas as pd
%cd UT03

[Errno 2] No such file or directory: 'UT03'
/home/jovyan/work/UT03


# CSV

In [28]:
dfNorte = pd.read_csv("ventas_norte_v2.csv", sep=";")
dfNorte = dfNorte.rename(columns={
                            "ID_Pedido": "id_transaccion", 
                            "Fecha_Transaccion": "fecha", 
                            "Producto": "producto", 
                            "Unidades": "cantidad",
                            "Total_Factura": "total_venta",
                            "Direccion_Envio": "ciudad"
                            }
                        )
dfNorte["region"] = "Norte"
dfNorte = dfNorte.drop(columns=["Cliente_Nombre", "Precio_Unitario"], axis=1)
dfNorte["total_venta"] = dfNorte["total_venta"].str.replace("$", "")
dfNorte["total_venta"] = pd.to_numeric(dfNorte["total_venta"])
dfNorte.head()

,id_transaccion,fecha,ciudad,producto,cantidad,total_venta,region
0,N-1000,2023-05-10 03:00:00,"Gran Vía, 49, Piso 4",Laptop Gamer,1,970.0,Norte
1,N-1001,2023-06-18 09:00:00,"Av. Libertad, 30, Piso 4",Mouse Ergonómico,4,3708.0,Norte
2,N-1002,2023-03-10 05:00:00,"Av. Libertad, 98, Piso 2",Monitor 4K,2,2820.0,Norte
3,N-1003,2023-05-10 03:00:00,"Paseo Gracia, 94, Piso 7",Webcam HD,2,2538.0,Norte
4,N-1004,2023-02-25 17:00:00,"Calle Mayor, 80, Piso 8",Webcam HD,2,908.0,Norte


# XLSX

In [34]:
dfSur = pd.read_excel("ventas_sur_v2.xlsx" , sheet_name=None)
dfSur = pd.concat(dfSur.values(), ignore_index=True)

dfSur = dfSur.rename(columns={
                            "Ref_Venta": "id_transaccion", 
                            "Fecha_Alta": "fecha", 
                            "Articulo": "producto", 
                            "Cantidad": "cantidad"
                            }
                        )
dfSur["region"] = "Sur"
dfSur["ciudad"] = ""
dfSur["total_venta"] = (dfSur["Precio_Base"] * dfSur["cantidad"] * (1 - dfSur["Descuento_Aplicado"]))
dfSur = dfSur.drop(columns=["Precio_Base", "Descuento_Aplicado"])
dfSur.head()

,id_transaccion,fecha,producto,cantidad,Es_Cliente_Corporativo,Estado_Envio,region,ciudad,total_venta
0,S-5000,2023-02-11 00:00:00,Webcam HD,9,False,Enviado,Sur,,3060.864
1,S-5001,2023-02-03 12:00:00,Teclado Mecánico,8,True,Devuelto,Sur,,4796.800
2,S-5002,2023-04-06 07:00:00,Mouse Ergonómico,6,False,Completado,Sur,,5828.160
3,S-5003,2023-04-02 05:00:00,Monitor 4K,1,True,Pendiente,Sur,,719.694
4,S-5004,2023-06-17 12:00:00,Mouse Ergonómico,2,True,Devuelto,Sur,,1177.981


# JSON

In [51]:
import json

with open('ventas_este_v2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

dfEste = pd.json_normalize(data)

columnas_deseadas = {
    'data.id_registro': 'id_transaccion',
    'data.payload.fecha_evento': 'fecha',
    'data.payload.comprador.ubicacion.ciudad': 'ciudad',
    'data.payload.transaccion.detalles_producto.nombre_comercial': 'producto',
    'data.payload.transaccion.cantidad_comprada': 'cantidad',
    'data.payload.transaccion.detalles_producto.precio_lista': 'precio_lista',
    'data.payload.transaccion.detalles_producto.impuestos.monto_iva': 'monto_iva'
}



dfEste = dfEste[list(columnas_deseadas.keys())].rename(columns=columnas_deseadas)
dfEste['fecha'] = dfEste['fecha'].str.replace(" 00:00:00", "")
dfEste["region"] = "Este"
dfEste.head()

,id_transaccion,fecha,ciudad,producto,cantidad,precio_lista,monto_iva,region
0,E-8000,2023-06-29,Sevilla,Monitor 4K,1,503,105.63,Este
1,E-8001,2023-04-12,Sevilla,Webcam HD,1,1122,235.62,Este
2,E-8002,2023-04-20,Sevilla,Webcam HD,2,994,208.74,Este
3,E-8003,2023-04-14,Madrid,Webcam HD,2,1255,263.55,Este
4,E-8004,2023-01-03,Sevilla,Monitor 4K,2,1458,306.18,Este


In [54]:
dfs = [dfSur, dfNorte, dfEste]

dfFinal = pd.concat(dfs, ignore_index=True)
dfFinal.head()

,id_transaccion,fecha,producto,cantidad,Es_Cliente_Corporativo,Estado_Envio,region,ciudad,total_venta,precio_lista,monto_iva
0,S-5000,2023-02-11 00:00:00,Webcam HD,9,False,Enviado,Sur,,3060.864,NaN,NaN
1,S-5001,2023-02-03 12:00:00,Teclado Mecánico,8,True,Devuelto,Sur,,4796.800,NaN,NaN
2,S-5002,2023-04-06 07:00:00,Mouse Ergonómico,6,False,Completado,Sur,,5828.160,NaN,NaN
3,S-5003,2023-04-02 05:00:00,Monitor 4K,1,True,Pendiente,Sur,,719.694,NaN,NaN
4,S-5004,2023-06-17 12:00:00,Mouse Ergonómico,2,True,Devuelto,Sur,,1177.981,NaN,NaN
